In [ ]:
# %%

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config

import numpy as np

from sklearnex import patch_sklearn
patch_sklearn()

label = 'signature'

data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'

train_data = TabularDataset(f'{data_url}train.csv')
test_data = TabularDataset(f'{data_url}test.csv')

train_data.head()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


In [ ]:
# %%

custom_hyperparameters = get_hyperparameter_config('zeroshot')
custom_hyperparameters['CAT'] = [{}]

custom_hyperparameters['LR'] = [
    {'multi_class':'multinomial', 'penalty':None, 'tol':1e-6, 'max_iter':10000,
        'ag_args': {'name_suffix': 'Base'}},
    {'multi_class':'multinomial', 'penalty':'l2', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Ridge'}},
    {'multi_class':'multinomial', 'penalty':'l1', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Lasso'}},
    {'multi_class':'multinomial', 'penalty':'elasticnet', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'ElasticNet'}}
    ]

custom_preset = {'auto_stack': True, 'dynamic_stacking': False,
                'hyperparameters':custom_hyperparameters, 'refit_full': False,
                'set_best_to_refit_full': False, 'save_bag_folds': True, 'num_bag_folds': 5}

In [ ]:
# %%

np.random.seed(2024)

predictor = TabularPredictor(label=label, problem_type='multiclass', eval_metric='log_loss', log_to_file=True)

predictor.fit(train_data, presets=custom_preset, excluded_model_types=['KNN'])  

No path specified. Models will be saved in: "AutogluonModels\ag-20240823_152547"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       22.66 GB / 31.75 GB (71.4%)
Disk Space Avail:   790.17 GB / 952.44 GB (83.0%)
Presets specified: [{'auto_stack': True, 'dynamic_stacking': False, 'hyperparameters': {'NN_TORCH': [{}, {'activation': 'elu', 'dropout_prob': 0.10077639529843717, 'hidden_size': 108, 'learning_rate': 0.002735937344002146, 'num_layers': 4, 'use_batchnorm': True, 'weight_decay': 1.356433327634438e-12, 'ag_args': {'name_suffix': '_r79', 'priority': -2}}, {'activation': 'elu', 'dropout_prob': 0.11897478034205347, 'hidden_size': 213, 'learning_rate': 0.0010474382260641949, 'num_layers': 4, 'use_batchnorm': False, 'weight_decay': 5.594471067786272e-10, 'ag_args': {'name_suf

In [ ]:
# %%

metrics = ['model', 'score_test', 'score_val', 'eval_metric', 'pred_time_test', 'fit_time']

df_leaders = predictor.leaderboard(test_data)

df_leaders.head(40)[metrics]

,model,score_test,score_val,eval_metric,pred_time_test,fit_time
0,WeightedEnsemble_L3,-0.118971,-0.113892,log_loss,123.351424,6233.392579
1,LightGBM_r96_BAG_L2,-0.120832,-0.123314,log_loss,118.399652,6061.330442
2,LightGBM_r94_BAG_L2,-0.122201,-0.124393,log_loss,117.934423,5994.520942
3,CatBoost_BAG_L2,-0.122445,-0.125301,log_loss,117.629385,10697.476008
4,NeuralNetFastAI_r134_BAG_L2,-0.124057,-0.127807,log_loss,118.408935,6004.161947
5,NeuralNetFastAI_r156_BAG_L2,-0.124575,-0.124044,log_loss,118.333115,5981.070942
6,NeuralNetFastAI_r102_BAG_L2,-0.125103,-0.127888,log_loss,118.281984,5969.924936
7,LightGBM_r30_BAG_L2,-0.126060,-0.128709,log_loss,120.154974,6625.563243
8,NeuralNetFastAI_r127_BAG_L2,-0.126517,-0.129654,log_loss,118.358085,5971.368936
9,XGBoost_r31_BAG_L2,-0.126746,-0.130517,log_loss,120.581290,7645.094577


In [ ]:
# %%

df_leaders.tail(20)[metrics]

,model,score_test,score_val,eval_metric,pred_time_test,fit_time
168,NeuralNetFastAI_r11_BAG_L1,-0.232891,-0.274309,log_loss,0.775334,36.744010
169,ExtraTreesGini_BAG_L1,-0.249652,-0.249207,log_loss,0.246883,0.823010
170,ExtraTrees_r49_BAG_L1,-0.249652,-0.249207,log_loss,0.324999,1.097006
171,ExtraTreesEntr_BAG_L1,-0.250766,-0.249721,log_loss,0.234881,0.776109
172,NeuralNetFastAI_r160_BAG_L1,-0.259701,-0.266490,log_loss,0.701162,22.818060
173,NeuralNetFastAI_r111_BAG_L1,-0.261511,-0.271522,log_loss,0.158829,5.655001
174,NeuralNetTorch_r36_BAG_L1,-0.275104,-0.280276,log_loss,0.172221,124.270094
175,LightGBM_r42_BAG_L2,-0.276767,-0.421863,log_loss,117.823875,6015.979429
176,NeuralNetFastAI_r143_BAG_L1,-0.293407,-0.295606,log_loss,0.190037,8.719002
177,NeuralNetFastAI_r156_BAG_L1,-0.298563,-0.299323,log_loss,0.133029,10.266001


In [ ]:
# %%

y_pred = predictor.predict_proba(test_data.drop(columns=[label]))
y_pred.head()

# predictor.evaluate(test_data, silent=True)

,-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12
0,0.000014,0.000010,0.000005,0.000068,0.999642,0.000183,0.000021,0.000015,0.000009,0.000006,0.000006,0.000010,0.000011
1,0.000044,0.000071,0.000083,0.000114,0.001020,0.316717,0.679782,0.001520,0.000140,0.000090,0.000294,0.000069,0.000056
2,0.000026,0.000029,0.000021,0.000016,0.000016,0.001415,0.998002,0.000376,0.000012,0.000013,0.000023,0.000024,0.000027
3,0.000016,0.000015,0.000013,0.000008,0.000013,0.000024,0.000031,0.000553,0.998986,0.000297,0.000011,0.000014,0.000018
4,0.000018,0.000019,0.000013,0.000014,0.000022,0.000040,0.001841,0.997737,0.000224,0.000027,0.000017,0.000015,0.000014


In [ ]:
# %%

# "AutogluonModels\ag-20240821_072221"